In [1]:
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ollama
import os

# Function to Generate AI Insights using Ollama
def generate_ai_insights(df_summary):
    prompt = f"Analyze the dataset summary and provide insights:\n\n{df_summary}"
    
    try:
        response = ollama.chat(
            model="gemma3:1b",  # ✅ CHANGE MODEL NAME HERE
            messages=[{"role": "user", "content": prompt}]
        )
        return response['message']['content']
    except Exception as e:
        return f"Error generating AI insights: {e}"

# Function to Generate Data Visualizations
def generate_visualizations(df):
    plot_paths = []

    # Ensure plots folder exists
    if not os.path.exists("plots"):
        os.makedirs("plots")
    
    # Histograms for numeric columns
    for col in df.select_dtypes(include=['number']).columns:
        plt.figure(figsize=(6, 4))
        sns.histplot(df[col], bins=30, kde=True, color="blue")
        plt.title(f"Distribution of {col}")
        path = f"plots/{col}_distribution.png"
        plt.savefig(path)
        plot_paths.append(path)
        plt.close()
    
    # Correlation Heatmap
    numeric_df = df.select_dtypes(include=['number'])
    if not numeric_df.empty:
        plt.figure(figsize=(8, 5))
        sns.heatmap(numeric_df.corr(), annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
        plt.title("Correlation Heatmap")
        path = "plots/correlation_heatmap.png"
        plt.savefig(path)
        plot_paths.append(path)
        plt.close()
    
    return plot_paths

# Function to Perform EDA
def eda_analysis(file_path):
    try:
        df = pd.read_csv(file_path)

        # Fill missing numeric values with median
        for col in df.select_dtypes(include=['number']).columns:
            df[col].fillna(df[col].median(), inplace=True)

        # Fill missing categorical values with mode
        for col in df.select_dtypes(include=['object']).columns:
            if not df[col].mode().empty:
                df[col].fillna(df[col].mode()[0], inplace=True)

        # Generate summary and missing value info
        summary = df.describe(include='all').to_string()
        missing_values = df.isnull().sum().to_string()

        # Generate insights
        insights = generate_ai_insights(summary)

        # Generate plots
        plot_paths = generate_visualizations(df)

        return f"✅ Data Loaded Successfully!\n\n📌 Summary:\n{summary}\n\n❓ Missing Values:\n{missing_values}\n\n🧠 AI Insights:\n{insights}", plot_paths

    except Exception as e:
        return f"❌ Error processing file: {e}", []

# Gradio UI
demo = gr.Interface(
    fn=eda_analysis,
    inputs=gr.File(type="filepath"),
    outputs=[gr.Textbox(label="📋 EDA Report"), gr.Gallery(label="📊 Data Visualizations")],
    title="📊 LLM-Powered Exploratory Data Analysis (EDA)",
    description="Upload any CSV dataset to get automatic EDA insights powered by AI (Ollama) and data visualizations.",
)

# Launch App
demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://2b3013c325d9c0eb50.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
